In [1]:
import os
from datasets import *

In [2]:
directory='/projects/ksun3/dwu25/repairllama/results/defects4j/repairllama/fft/patches'

In [3]:
def extract_added_lines(diff):
    # 分割diff字符串为单独的行
    lines = diff.split('\n')
    # 提取所有以'+'开始的行
    added_lines =[line.replace('+', '')+'\n' for line in lines[2:] if line.startswith('+')]
    return ''.join(added_lines)

# Diff文件的内容
diff_content = """
public LegendItemCollection getLegendItems() {
         LegendItemCollection result = new LegendItemCollection();
         if (this.plot == null) {
             return result;
         }
         int index = this.plot.getIndexOf(this);
         CategoryDataset dataset = this.plot.getDataset(index);
-        if (dataset != null) {
+        if (dataset == null) {
+            return result;
         }
         int seriesCount = dataset.getRowCount();
         if (plot.getRowRenderingOrder().equals(SortOrder.ASCENDING)) {
             for (int i = 0; i < seriesCount; i++) {
                 if (isSeriesVisibleInLegend(i)) {
                     LegendItem item = getLegendItem(index, i);
                     if (item != null) {
                         result.add(item);
                     }
                 }
             }
         } 
"""

# 获取添加的行
added_lines = extract_added_lines(diff_content)

# 打印结果
added_lines



'        if (dataset == null) {\n            return result;\n'

In [4]:
sample_list=[]
for entry in os.listdir(directory):
    sample={}
    path = os.path.join(directory, entry)
    if os.path.isdir(path):
        with open(path+'/prompt.txt', 'r') as file:
            sample['input']=file.read()
        with open(path+'/target.diff', 'r') as file:
            target=file.read()
        sample['bug_id']=entry
        sample['target']=extract_added_lines(target)
        sample_list.append(sample)

In [5]:
import os
import json

# 假设 directory 是你的目录路径
output_file_path = 'defects4j_with_buggyline.jsonl'

with open(output_file_path, 'w') as output_file:
    for entry in os.listdir(directory):
        sample = {}
        path = os.path.join(directory, entry)
        if os.path.isdir(path):
            with open(os.path.join(path, 'prompt.txt'), 'r') as file:
                sample['input'] = file.read()
            with open(os.path.join(path, 'target.diff'), 'r') as file:
                target = file.read()
            sample['bug_id'] = entry
            sample['target'] = extract_added_lines(target)  # 假设这个函数已经被正确定义
            
            # 将sample字典转换为JSON字符串，并写入到.jsonl文件中
            json_line = json.dumps(sample)
            output_file.write(json_line + '\n')


In [5]:
dataset = Dataset.from_list(sample_list)

In [6]:
dataset.save_to_disk('defects4j_with_buggy_repairllama')

Saving the dataset (0/1 shards):   0%|          | 0/479 [00:00<?, ? examples/s]